In [35]:
import requests
import time
from selenium import webdriver
from PIL import Image as pil

In [36]:
category = {
    100 : '정치',
    101 : '경제',
    102 : '사회',
    103 : '생활/문화',
    104 : '세계',
    105 : 'IT/과학'
}

In [37]:
category, date, page = 100, 20201029, 1

url = 'https://news.naver.com/main/list.nhn?\
mode=LSD&mid=sec&sid1={}&date={}&page={}'.format(category, date, page)

In [38]:
driver = webdriver.Chrome()
driver.get(url)

In [44]:
articles = driver.find_elements_by_css_selector(".list_body > ul > li")
len(articles)

20

In [45]:
title = articles[0].find_element_by_css_selector("dt:not(.photo) > a").text
link = articles[0].find_element_by_css_selector("dt:not(.photo) > a")\
.get_attribute("href")
title, link

('문대통령, WHO총장에 "K-방역 호평 감사"',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=422&aid=0000454408')

In [41]:
from bs4 import BeautifulSoup

In [46]:
response = requests.get(link)
dom = BeautifulSoup(response.content, "html.parser")

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
content = dom.select_one('#articleBodyContents').text.replace('\n', "").replace('- ', '').split('{}')[1].strip()

In [47]:
def get_articles(category, date, page):
    url = 'https://news.naver.com/main/list.nhn?\
mode=LSD&mid=sec&sid1={}&date={}&page={}'.format(category, date, page)
    driver = webdriver.Chrome()
    driver.get(url)
    articles = driver.find_elements_by_css_selector('.list_body.newsflash_body > ul > li')
    datas = []
    for article in articles:
        title = articles[0].find_element_by_css_selector('dt:not(.photo) > a').text
        link = articles[0].find_element_by_css_selector('dt:not(.photo) > a').get_attribute('href')
        response = requests.get(link)
        dom = BeautifulSoup(response.content, "html.parser")
        content = dom.select_one('#articleBodyContents').text.replace('\n', "").replace('- ', '').split('{}')[1].strip()
        datas.append({
            "title" : title,
            "link" : link,
            "content": content,
            "category": category
        })
    driver.quit()
    return pd.DataFrame(datas)

In [48]:
get_articles(100, 20201028, 1)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
dfs = []
for category in categories:
    print(categories[category], end=" ")
    for page in range(1, 6):
        print(page, end=" ")
        df = get.article(category, date, page)
        dfs.append(df)
    print()
articles_df = pd.concat(dfs, ignore_index=True)
articles_df.tail()

In [ ]:
articles_df.to_csv("article.csv", index=False, encoding="utf-8-sig")